In [3]:
fashionmnist_data_rtime = [
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.9027},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.9006},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.8908},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.9012},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.8997},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.896},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.9005},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.8987},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.8968},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.8978},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.8974},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.8938},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.8976},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.8968},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.8953},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.894},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.8922},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.8938},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.8982},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.86},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.7664},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.9023},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.8966},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.8975},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.8987},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.8844},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.8978},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.901},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.8982},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.8972},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.8949},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.8793},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.879},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.8958},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.8932},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.8981},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.7323},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.7121},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.7071},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.9028},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.8963},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.8982},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.8057},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.7584},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.8621},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.899},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.9002},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.8962},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.8666},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.879},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.8913},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.8923},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.8953},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.894},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.711},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.7096},
  {"total_clients": 4, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.709},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.9027},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.8974},
  {"total_clients": 4, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.8948},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.7099},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.7113},
  {"total_clients": 5, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.8298},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.8981},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.8996},
  {"total_clients": 5, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.8957},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.7117},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.7222},
  {"total_clients": 6, "rounds": 1, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.7069},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.8922},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.8909},
  {"total_clients": 6, "rounds": 2, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.8881}
]

fashionmnist_data = [
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.914},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.913},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.9132},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.9117},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.91},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.9118},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 0, "accuracy": 0.9101},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 0, "accuracy": 0.9067},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 0, "accuracy": 0.9078},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.9104},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.9092},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.9084},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.9108},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.9093},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.9058},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 1, "accuracy": 0.9115},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 1, "accuracy": 0.9072},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 1, "accuracy": 0.9094},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.8976},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.908},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.9096},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.9042},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.9039},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.8959},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 2, "accuracy": 0.9062},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 2, "accuracy": 0.9045},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 2, "accuracy": 0.9028},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.7859},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.8727},
  {"total_clients": 5, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.9011},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.8955},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.9018},
  {"total_clients": 7, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.9049},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 100, "byzantine_clients": 3, "accuracy": 0.9063},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 80, "byzantine_clients": 3, "accuracy": 0.8932},
  {"total_clients": 10, "rounds": 5, "clients_per_round_percent": 70, "byzantine_clients": 3, "accuracy": 0.8986}
]

# Analisi all'aumento del numero di nodi totali nei vari scenari

In [37]:
def get_by_client_selection(client_selection, ds, num_clients, rounds) -> list[list[dict[str, any]]]:
    cs = \
    [
        [
            {
                "Accuracy": x["accuracy"], 
                "Byzantine": x["byzantine_clients"],
                "Total Clients": x["total_clients"],
                "Rounds": x["rounds"]
            }
            for x in ds
            if x["total_clients"] == total and 
            x["rounds"] == rnd and 
            x["clients_per_round_percent"] == client_selection
        ]
        for total in num_clients
        for rnd in rounds
    ]

    cs.sort(key=lambda x : x[0]['Byzantine'])

    return cs

def get_seq(data: list[list[dict[str, any]]]) -> list[str]:
    return [
        "\n" + str(group[0]['Total Clients']) + " clients, " + str(group[0]['Rounds']) + " rounds: " +
        ' -> '.join([str(x['Accuracy']) for x in group])
        for group in data
    ]


def get_performance_decrease(client_selection, ds, num_clients, rounds):
    cs = get_by_client_selection(client_selection, ds, num_clients, rounds)
    seq = get_seq(cs)
    return seq

In [50]:
for x in get_performance_decrease(100, fashionmnist_data_rtime, [4,5,6], [1, 2]):
    print(x)


4 clients, 1 rounds: 0.9027 -> 0.8982 -> 0.7323 -> 0.711

4 clients, 2 rounds: 0.9012 -> 0.9023 -> 0.9028 -> 0.9027

5 clients, 1 rounds: 0.9005 -> 0.8987 -> 0.8057 -> 0.7099

5 clients, 2 rounds: 0.8978 -> 0.901 -> 0.899 -> 0.8981

6 clients, 1 rounds: 0.8976 -> 0.8949 -> 0.8666 -> 0.7117

6 clients, 2 rounds: 0.894 -> 0.8958 -> 0.8923 -> 0.8922



4 clients, 1 rounds: 0.9027 -> 0.8982 -> 0.7323 -> 0.711
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

4 clients, 2 rounds: 0.9012 -> 0.9023 -> 0.9028 -> 0.9027
Abbattimento praticamente nullo

5 clients, 1 rounds: 0.9005 -> 0.8987 -> 0.8057 -> 0.7099
Abbattimento delle performance minore rispetto al caso precedente ma comunque significativo: 9% all'introduzione del secondo nodo bizantino, 10% al terzo.

5 clients, 2 rounds: 0.8978 -> 0.901 -> 0.899 -> 0.8981
Abbattimento praticamente nullo 

6 clients, 1 rounds: 0.8976 -> 0.8949 -> 0.8666 -> 0.7117
Prestazioni leggermente più basse inizialmente, ma più robusto. Solo al terzo nodo bizantino si ha un abbattimento significativo delle performance.

6 clients, 2 rounds: 0.894 -> 0.8958 -> 0.8923 -> 0.8922
Abbattimento praticamente nullo

L'aumento del numero di round non sembra influenzare le performance, anzi, sembra migliorare la robustezza del sistema.

In [51]:
for x in get_performance_decrease(80, fashionmnist_data_rtime, [4,5,6], [1, 2]):
    print(x)


4 clients, 1 rounds: 0.9006 -> 0.86 -> 0.7121 -> 0.7096

4 clients, 2 rounds: 0.8997 -> 0.8966 -> 0.8963 -> 0.8974

5 clients, 1 rounds: 0.8987 -> 0.8844 -> 0.7584 -> 0.7113

5 clients, 2 rounds: 0.8974 -> 0.8982 -> 0.9002 -> 0.8996

6 clients, 1 rounds: 0.8968 -> 0.8793 -> 0.879 -> 0.7222

6 clients, 2 rounds: 0.8922 -> 0.8932 -> 0.8953 -> 0.8909


4 client, 1 round -> 0.9006, 0.86, 0.7121, 0.7096   
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

5 client, 1 round -> 0.8987, 0.8844, 0.7584, 0.7113 
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

6 client, 1 round -> 0.8968, 0.8793, 0.879, 0.7222 
Prestazioni leggermente più basse inizialmente, ma più robusto. Solo al terzo nodo bizantino si ha un abbattimento significativo delle performance.

4 client, 2 round -> 0.8997, 0.8966, 0.8963, 0.8974 
Abbattimento praticamente nullo

5 client, 2 round -> 0.8974, 0.8982, 0.9002, 0.8996 
Abbattimento praticamente nullo

6 client, 2 round -> 0.8922, 0.8932, 0.8953, 0.8909 
Abbattimento praticamente nullo

In [52]:
for x in get_performance_decrease(70, fashionmnist_data_rtime, [4,5,6], [1, 2]):
    print(x)


4 clients, 1 rounds: 0.8908 -> 0.7664 -> 0.7071 -> 0.709

4 clients, 2 rounds: 0.896 -> 0.8975 -> 0.8982 -> 0.8948

5 clients, 1 rounds: 0.8968 -> 0.8978 -> 0.8621 -> 0.8298

5 clients, 2 rounds: 0.8938 -> 0.8972 -> 0.8962 -> 0.8957

6 clients, 1 rounds: 0.8953 -> 0.879 -> 0.8913 -> 0.7069

6 clients, 2 rounds: 0.8938 -> 0.8981 -> 0.894 -> 0.8881


4 client, 1 round -> 0.8908, 0.7664, 0.7071, 0.709 
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

5 client, 1 round -> 0.8968, 0.8978, 0.8621, 0.8298 
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

6 client, 1 round -> 0.8953, 0.879, 0.8913, 0.7069 
Prestazioni leggermente più basse inizialmente, ma più robusto. Solo al terzo nodo bizantino si ha un abbattimento significativo delle performance.

4 client, 2 round -> 0.896, 0.8975, 0.8982, 0.8948 
Abbattimento praticamente nullo

5 client, 2 round -> 0.8938, 0.8972, 0.8962, 0.8957 
Abbattimento praticamente nullo

6 client, 2 round -> 0.8938, 0.8981, 0.894, 0.8881  
Abbattimento praticamente nullo

# Analisi all'aumentare del numero di round

In [53]:
from numpy import mean

def mean_at_byzantine_n(n, n_round, client_selection):
    l = []

    for x in fashionmnist_data_rtime:
        if x['rounds'] == n_round and x['clients_per_round_percent'] == client_selection and x['byzantine_clients'] == n-1:
            for y in fashionmnist_data_rtime:
                if y['rounds'] == n_round and y['clients_per_round_percent'] == client_selection and y['byzantine_clients'] == n and x['total_clients'] == y['total_clients']:
                    l.append(x['accuracy']-y['accuracy'])
    return mean(l)                

In [54]:
print("Mean performance decrease: Single Round, Client Selection 100%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 100))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 100))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 100))

Mean performance decrease: Single Round, Client Selection 100%
First byzantine: 0.0029999999999999285
Second byzantine: 0.09573333333333338
Third byzantine: 0.09066666666666667


In [55]:
print("Mean performance decrease: Double Round, Client Selection 100%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 100))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 100))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 100))

Mean performance decrease: Double Round, Client Selection 100%
First byzantine: -0.0020333333333333314
Second byzantine: 0.001666666666666668
Third byzantine: 0.0003666666666667003


In [56]:
print("Mean performance decrease: Single Round, Client Selection 80%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 80))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 80))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 80))

Mean performance decrease: Single Round, Client Selection 80%
First byzantine: 0.024133333333333378
Second byzantine: 0.0914
Third byzantine: 0.06879999999999997


In [57]:
print("Mean performance decrease: Double Round, Client Selection 80%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 80))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 80))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 80))

Mean performance decrease: Double Round, Client Selection 80%
First byzantine: 0.0004333333333333596
Second byzantine: -0.0012666666666666753
Third byzantine: 0.001300000000000005


In [58]:
print("Mean performance decrease: Single Round, Client Selection 70%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 70))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 70))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 70))

Mean performance decrease: Single Round, Client Selection 70%
First byzantine: 0.04656666666666668
Second byzantine: 0.0275666666666667
Third byzantine: 0.0716


In [59]:
print("Mean performance decrease: Double Round, Client Selection 70%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 70))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 70))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 70))

Mean performance decrease: Double Round, Client Selection 70%
First byzantine: -0.003066666666666625
Second byzantine: 0.0014666666666666532
Third byzantine: 0.00326666666666664
